In [1]:
# need python 3.12+ for typing
!python -V

Python 3.12.6


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.sokoban_solver import SokobanEdge, SokobanNode, SokobanSolver

**Solver to do**
- Clean up the solver
    - Make the solver have an internal solving state that it updates during solving
    - Have a solve_step() method as the "core" thing, with a solve_run() that loops things
    - You can specify a timeout or max steps on the solve_run()
    - And maybe also the option to print debug stuff with rate limiting?
- Create constructors for Board objects, ideally something interactive! Like, a lightweight web app that serves as an editor.
- Come up with a format for putting problems in jsons
- Get the solver working on the "11" problem
    - "Working" = "In well under 1 second"
    - Maybe start with a few other demo puzzles
    - Might want the ability to create heuristics? Like by subclassing to specify how the queue is implemented.
    - So replace the queue list with a generic queue object (priority queue?...) that different solver implement differently.
    - Ex. heuristic: past a depth of len(board.boxes)?
    - See import queue; queue.PriorityQueue
- Create unit tests (runs solver on some test problems)

**Generator to do**
- Learn the papers
    - https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=af6c8e280946f019c7f632c8281a35ddf1fdc9b0
    - https://ianparberry.com/techreports/LARC-2011-01.pdf
    - Also do a bit more literature search
- Start implementing techniques

## Basic demos

In [4]:
# demo
airs = {(0, 0), (0, 1), (0, 2), (0, 3), (1, 3)}
boxes = {(0, 2)}
goals = {(0, 2)}
player = (0, 0)

node1 = SokobanNode(airs, boxes, goals, player)
print('Player component: ', node1.player_component)
print('Avaiable pushes: ', node1.edges)
print('Is solved?: ', node1.solved)

print(node1)

Player component:  {(0, 1), (0, 0)}
Avaiable pushes:  [SokobanEdge((0, 1), (0, 1), (0, 2), (0, 3))]
Is solved?:  True
█ █ █ █ █ █
█ █ █ █ █ █
█ █     █ █
█ █ ■ █ █ █
█ █   █ █ █
█ █ ○ █ █ █
█ █ █ █ █ █
█ █ █ █ █ █


In [5]:
edge = SokobanEdge((0, 1), (0, 1), (0, 2), (0, 3))
node2 = node1 + edge
print('Is solved?: ', node2.solved)
print(node2)

Is solved?:  False
█ █ █ █ █ █
█ █ █ █ █ █
█ █ □   █ █
█ █ ● █ █ █
█ █   █ █ █
█ █   █ █ █
█ █ █ █ █ █
█ █ █ █ █ █


In [6]:
node1 == node1

True

In [7]:
node1 == node2

False

In [8]:
# epic
airs = {(0, 0), (0, 1), (0, 2), (0, 3), (1, 3)}
boxes = {(0, 2)}
goals = {(0, 2)}

player1 = (0, 0)
player2 = (0, 1)

node3 = SokobanNode(airs, boxes, goals, player1)
node4 = SokobanNode(airs, boxes, goals, player2)

node3 == node4

True

In [9]:
# trivial puzzle
airs = {(0, 0), (0, 1), (0, 2)}
boxes = {(0, 2)}
goals = {(0, 2)}
player = (0, 0)

node = SokobanNode(airs, boxes, goals, player)
solver = SokobanSolver(node)
solver.solve()

200

In [10]:
print(solver.solution)

█ █ █ █ █
█ █ █ █ █
█ █ ■ █ █
█ █   █ █
█ █ ○ █ █
█ █ █ █ █
█ █ █ █ █

with no moves


In [11]:
# one-push puzzle
airs = {(0, 0), (0, 1), (0, 2)}
boxes = {(0, 1)}
goals = {(0, 2)}
player = (0, 0)

node = SokobanNode(airs, boxes, goals, player)
solver = SokobanSolver(node)
solver.solve()

200

In [12]:
print(solver.solution)

█ █ █ █ █
█ █ █ █ █
█ █ · █ █
█ █ □ █ █
█ █ ○ █ █
█ █ █ █ █
█ █ █ █ █

to

█ █ █ █ █
█ █ █ █ █
█ █ ■ █ █
█ █ ○ █ █
█ █   █ █
█ █ █ █ █
█ █ █ █ █

via

Move (0, 1) from (0, 0) to (0, 1)


In [13]:
# simple puzzle
airs = {
    (1, 3), (2, 3), (3, 3),
    (1, 2), (2, 2), (3, 2),
    (1, 1), (2, 1), (3, 1),
}
boxes = {(1, 2), (2, 1), (2, 2)}
goals = {(1, 3), (3, 1), (2, 3)}
player = (1, 1)

node = SokobanNode(airs, boxes, goals, player)
solver = SokobanSolver(node)
solver.solve()

200

In [14]:
print(solver.solution)

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ · ·   █ █
█ █ □ □   █ █
█ █ ○ □ · █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █

to

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ ■ ■   █ █
█ █   ○   █ █
█ █     ■ █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █

via

Move (0, 1) from (1, 1) to (1, 2)
Move (1, 0) from (1, 1) to (2, 1)
Move (0, 1) from (2, 1) to (2, 2)


## Real puzzles

https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=1

In [15]:
airs_arr = [
    [1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 1, 1, 1, 1, 1],
    [0, 1, 1, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 0, 0, 1],
    [0, 1, 1, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 0, 0],
]
airs_arr = list(reversed(airs_arr))

airs = set()
for i in range(len(airs_arr[0])):
    for j in range(len(airs_arr)):
        if airs_arr[j][i] == 1:
            airs.add((i+1, j+1))

boxes = {(2, 3), (3, 3)}
goals = {(3, 3), (3, 8)}
player = (3, 2)

node = SokobanNode(airs, boxes, goals, player)
solver = SokobanSolver(node)
solver.solve()

200

In [16]:
print(solver.solution)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █       █ █ █ █ █ █ █ █
█ █   █ · █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █       █           █ █
█ █ █     █   █ █ █   █ █
█ █   □ ■       █ █   █ █
█ █ █   ○ █           █ █
█ █ █ █ █ █       █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █       █ █ █ █ █ █ █ █
█ █   █ ■ █ █ █ █ █ █ █ █
█ █   █ ○ █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █       █           █ █
█ █ █     █   █ █ █   █ █
█ █     ■       █ █   █ █
█ █ █     █           █ █
█ █ █ █ █ █       █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

via

Move (0, 1) from (2, 2) to (2, 3)
Move (0, 1) from (2, 3) to (2, 4)
Move (1, 0) from (1, 5) to (2, 5)
Move (0, 1) from (3, 4) to (3, 5)
Move (0, 1) from (3, 5) to (3, 6)
Move (0, 1) from (3, 6) to (3, 7)


https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=11

In [20]:
airs = {
    (4, 7),
    (2, 6), (3, 6), (4, 6), (5, 6), (6, 6), (7, 6),
    (2, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5),
    (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4),
    (1, 3), (2, 3), (3, 3), (4, 3),
    (1, 2), (4, 2),
    (1, 1), (2, 1), (3, 1), (4, 1),
}
boxes = {(3, 3), (3, 4), (2, 4), (5, 4), (5, 5), (7, 5)}
goals = {(1, 1), (1, 4), (3, 6), (5, 4), (6, 4), (6, 5)}
player = (2, 1)

node = SokobanNode(airs, boxes, goals, player)
solver = SokobanSolver(node)

In [21]:
# unable to find the solution in reasonable time
solver.solve(max_seconds=10)

301

In [22]:
solver.n_iters_overall

41896

In [23]:
solver.n_seconds_overall

10.000865936279297

## Bottom

solver.step()
print(len(solver.queue))
print()
print(solver.queue.get_str_end_nodes())